In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
#en_nlp = spacy.load('en')

Convert Json to Pandas Dataframe


In [2]:
data= pd.read_json("merged_file.json")

In [3]:
data.head

<bound method NDFrame.head of                                                   data
0    {'paragraphs': [{'qas': [{'question': 'In whic...
1    {'paragraphs': [{'qas': [{'question': 'What ar...
2    {'paragraphs': [{'qas': [{'question': 'What ar...
3    {'paragraphs': [{'qas': [{'question': 'In whic...
4    {'paragraphs': [{'qas': [{'question': 'What ar...
..                                                 ...
146  {'paragraphs': [{'qas': [{'question': 'Where i...
147  {'paragraphs': [{'qas': [{'question': 'Where i...
148  {'paragraphs': [{'qas': [{'question': 'Where i...
149  {'paragraphs': [{'qas': [{'question': 'Where i...
150  {'paragraphs': [{'qas': [{'question': 'Where i...

[151 rows x 1 columns]>

In [4]:
train=data[:100]
valid=data[100:150]

In [5]:
cols = ["text","question","answer","answer_start"]
comp_list = []
for index, row in train.iterrows():
    for i in range(len(row["data"]['paragraphs'])):
      for j in range(len(row["data"]['paragraphs'][i]['qas'])):
        temp_list = []
        temp_list.append(row["data"]['paragraphs'][i]['context'])
        temp_list.append(row["data"]['paragraphs'][i]['qas'][j]['question'])
        if row["data"]['paragraphs'][i]['qas'][j]['is_impossible']==False:
          temp_list.append(row["data"]['paragraphs'][i]['qas'][j]['answers'][0]['text'])
          temp_list.append(row["data"]['paragraphs'][i]['qas'][j]['answers'][0]['answer_start'])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns = cols) 
new_df

,text,question,answer,answer_start
0,Los Angeles is a sprawling Southern California...,In which state is Los Angeles?,California,36.0
1,Los Angeles is a sprawling Southern California...,What more can be told about the city?,TCL Chinese Theatre displays celebrities’ hand...,266.0
2,Los Angeles is a sprawling Southern California...,What are some iconic thing about the city?,"Paramount Pictures, Universal and Warner Brothers",161.0
3,New York City comprises 5 boroughs sitting whe...,What are some iconic thing about the city?,Empire State Building and sprawling Central Park,269.0
4,New York City comprises 5 boroughs sitting whe...,What could be said the geography of the city?,comprises 5 boroughs sitting where the Hudson ...,14.0
...,...,...,...,...
368,Boise is the capital city of Idaho. The Boise ...,What are some of the famous attractions in Boi...,"In an art deco building, downtown�s Boise Art ...",132.0
369,Fremont is a city in California's San Francisc...,Which state is the city in?,California,21.0
370,Fremont is a city in California's San Francisc...,Where is the city located?\n,California's San Francisco Bay Area,21.0
371,Fremont is a city in California's San Francisc...,What is the city?,Fremont,0.0


In [6]:
new_df.dropna(inplace=True)

In [7]:
new_df['answer_start'] = new_df['answer_start'].astype('int')

In [8]:
new_df

,text,question,answer,answer_start
0,Los Angeles is a sprawling Southern California...,In which state is Los Angeles?,California,36
1,Los Angeles is a sprawling Southern California...,What more can be told about the city?,TCL Chinese Theatre displays celebrities’ hand...,266
2,Los Angeles is a sprawling Southern California...,What are some iconic thing about the city?,"Paramount Pictures, Universal and Warner Brothers",161
3,New York City comprises 5 boroughs sitting whe...,What are some iconic thing about the city?,Empire State Building and sprawling Central Park,269
4,New York City comprises 5 boroughs sitting whe...,What could be said the geography of the city?,comprises 5 boroughs sitting where the Hudson ...,14
...,...,...,...,...
368,Boise is the capital city of Idaho. The Boise ...,What are some of the famous attractions in Boi...,"In an art deco building, downtown�s Boise Art ...",132
369,Fremont is a city in California's San Francisc...,Which state is the city in?,California,21
370,Fremont is a city in California's San Francisc...,Where is the city located?\n,California's San Francisco Bay Area,21
371,Fremont is a city in California's San Francisc...,What is the city?,Fremont,0


In [9]:
new_df.shape

(363, 4)

In [10]:
new_df.to_csv("data.csv", index = None)

Create dictionary of sentence embeddings for faster computation


In [11]:
paras = list(new_df["text"].drop_duplicates().reset_index(drop= True))

In [12]:
len(paras)

100

In [13]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [14]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [15]:
len(sentences)

370

In [16]:
!mkdir GloVe

In [17]:
!curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   353    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5190k      0  0:06:49  0:06:49 --:--:-- 5113k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


In [25]:
!mkdir fastText

In [26]:
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  20.7M      0  0:01:10  0:01:10 --:--:-- 21.6M
Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [ ]:
!mkdir encoder

In [19]:
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  39.9M      0  0:00:03  0:00:03 --:--:-- 39.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  35.2M      0  0:00:04  0:00:04 --:--:-- 35.2M


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from models import InferSent
V = 2
MODEL_PATH = '/content/encoder/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [30]:
W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [43]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import matplotlib

import numpy as np
import torch

GLOVE_PATH = '/content/GloVe/glove.840B.300d.txt'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
!python models.py

In [58]:
from models import InferSent
model_version = 1
MODEL_PATH = "/content/encoder/infersent1.pkl"
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [59]:
use_cuda = False
model = model.cuda() if use_cuda else model

In [60]:
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [61]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [ ]:
dict_embeddings = {}
for i in range(len(sentences)):
    print(i)
    dict_embeddings[sentences[i]] = model.encode([sentences[i]], tokenize=True)
    #dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [64]:
questions = list(new_df["question"])

In [65]:
len(questions)

363

In [69]:
for i in range(len(questions)):
    print(i)
    dict_embeddings[questions[i]] = model.encode([questions[i]], tokenize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [70]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}


In [71]:
d1

{'Los Angeles is a sprawling Southern California city and the center of the nation’s film and television industry.': array([[ 0.12785082,  0.23619145,  0.12586468, ...,  0.07998279,
         -0.03814263,  0.04452515]], dtype=float32),
 'On Hollywood Boulevard, TCL Chinese Theatre displays celebrities’ hand- and footprints, the Walk of Fame honors thousands of luminaries and vendors sell maps to stars’ homes New York City comprises 5 boroughs sitting where the Hudson River meets the Atlantic Ocean.': array([[ 0.14571707,  0.17790712,  0.1629303 , ...,  0.16522561,
         -0.02339158,  0.09974831]], dtype=float32),
 'Its iconic sites include skyscrapers such as the Empire State Building and sprawling Central Park.': array([[0.152361  , 0.11308052, 0.0921296 , ..., 0.06004412, 0.00242643,
         0.00715252]], dtype=float32),
 'Philadelphia, Pennsylvania’s largest city, is notable for its rich history, on display at the Liberty Bell, Independence Hall (where the Declaration of Independ

In [76]:
with open('Vineet/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [77]:
with open('Vineet/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)